## This is the api for the match from ICD code to UMLS code.

### Authentication

In [32]:
#!/usr/bin/python
## 6/16/2017 - remove PyQuery dependency
## 5/19/2016 - update to allow for authentication based on api-key, rather than username/pw
## See https://documentation.uts.nlm.nih.gov/rest/authentication.html for full explanation

import requests
#from pyquery import PyQuery as pq
import lxml.html as lh
from lxml.html import fromstring

uri="https://utslogin.nlm.nih.gov"
#option 1 - username/pw authentication at /cas/v1/tickets
auth_endpoint = "/cas/v1/tickets/"
#option 2 - api key authentication at /cas/v1/api-key
#auth_endpoint = "/cas/v1/api-key"

class Authentication:

   def __init__(self, username, password):
   #def __init__(self, apikey):
    self.username=username
    self.password=password
    #self.apikey=apikey
    self.service="http://umlsks.nlm.nih.gov"

   def gettgt(self):
     params = {'username': self.username,'password': self.password}
     #params = {'apikey': self.apikey}
     h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
     r = requests.post(uri+auth_endpoint,data=params,headers=h)
     response = fromstring(r.text)
     ## extract the entire URL needed from the HTML form (action attribute) returned - looks similar to 
     ## https://utslogin.nlm.nih.gov/cas/v1/tickets/TGT-36471-aYqNLN2rFIJPXKzxwdTNC5ZT7z3B3cTAKfSc5ndHQcUxeaDOLN-cas
     ## we make a POST call to this URL in the getst method
     tgt = response.xpath('//form/@action')[0]
     return tgt

   def getst(self,tgt):

     params = {'service': self.service}
     h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
     r = requests.post(tgt,data=params,headers=h)
     st = r.text
     return st

### Retrive the information

In [33]:
import json
import argparse
username = 'ChaoZhang'
password = '3f5hr7z9a0!!'
version = '2019AA'
#identifier = '79'
source = 'ICD9CM'
AuthClient = Authentication(username, password)

In [34]:
tgt = AuthClient.gettgt()
uri = "https://uts-ws.nlm.nih.gov"

In [48]:
def icd9tocui(identifier):
    content_endpoint = "/rest/search/2019AA?string="+str(identifier)+"&sabs=ICD9CM&searchType=exact&inputType=sourceUi"
    query = {'ticket':AuthClient.getst(tgt)}
    r = requests.get(uri+content_endpoint,params=query)
    r.encoding = 'utf-8'
    items  = json.loads(r.text)
    jsonData = items["result"]
    return jsonData['results'][0]['ui']

### Generate the icd9 to umls code matching file

In [59]:
import pandas as pd
MEDI = pd.read_csv('MEDI.csv')
Medi_indi = MEDI[['STR', 'CODE']]
for x in Medi_indi['CODE']:
    x = x.split('-')[0]
Medi_indi.columns = ['Medication', 'Indication']
Medi_indi.set_index('Indication', inplace = True)

In [68]:
i = 0
s = set(Medi_indi.index)
print(len(s))
dic = {}
for idx in s:
    dic[idx] = icd9tocui(idx)
    i += 1
    if i%1000==0: print(i)

2705
1000
2000


In [71]:
pd.Series(dic).to_csv('ICD9CM_to_UMLS.csv')